In [ ]:
# Step 1: Install of the packages
!pip install mercantile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# The first step is to define the min/max (lat,lng)
# Set the zoom level (resolution level)
lat_lng = [52.069271, 19.480322]
delta = 6
tl = [lat_lng[0]+delta * 0.55, lat_lng[1]-delta*0.95]
br = [lat_lng[0]-delta * 0.8, lat_lng[1]+delta]
z = 8 # Set the resolution

In [ ]:
# find the tile set IDs (x/y) for the top left and bottom right at the zoom level
import mercantile
tl_tiles = mercantile.tile(tl[1],tl[0],z)
br_tiles = mercantile.tile(br[1],br[0],z)

x_tile_range = [tl_tiles.x,br_tiles.x]
print(x_tile_range)
y_tile_range = [tl_tiles.y,br_tiles.y]
print(y_tile_range[0])

[137, 146]
80


In [ ]:
# Make the folders
!mkdir ./satellite_images
!rm -rf ./satellite_images/*
!mkdir ./elevation_images
!rm -rf ./elevation_images/*

In [ ]:
# Loop over the ranges and extract the images from mapbox for satellite at @2x resolution (512x512)
import requests
import shutil
for i,x in enumerate(range(x_tile_range[0],x_tile_range[1]+1)):
    for j,y in enumerate(range(y_tile_range[0],y_tile_range[1]+1)):
        print(x,y)
        r = requests.get('https://api.mapbox.com/v4/mapbox.satellite/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.png?access_token=pk.eyJ1Ijoibml6d2FudCIsImEiOiJjbGQ1djhudzMwZXZmM3FtbW1hc3V0dWZyIn0.DuLNzTVZ8mMUlZEa83OdRQ', stream=True)
        if r.status_code == 200:
            with open('./satellite_images/' + str(i) + '.' + str(j) + '.png', 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

137 80
137 81
137 82
137 83
137 84
137 85
137 86
137 87
137 88
137 89
138 80
138 81
138 82
138 83
138 84
138 85
138 86
138 87
138 88
138 89
139 80
139 81
139 82
139 83
139 84
139 85
139 86
139 87
139 88
139 89
140 80
140 81
140 82
140 83
140 84
140 85
140 86
140 87
140 88
140 89
141 80
141 81
141 82
141 83
141 84
141 85
141 86
141 87
141 88
141 89
142 80
142 81
142 82
142 83
142 84
142 85
142 86
142 87
142 88
142 89
143 80
143 81
143 82
143 83
143 84
143 85
143 86
143 87
143 88
143 89
144 80
144 81
144 82
144 83
144 84
144 85
144 86
144 87
144 88
144 89
145 80
145 81
145 82
145 83
145 84
145 85
145 86
145 87
145 88
145 89
146 80
146 81
146 82
146 83
146 84
146 85
146 86
146 87
146 88
146 89


In [ ]:
# Combine the tiles into a single large image
!mkdir ./composite_images
!mkdir ./animate

import PIL
import math
from PIL import Image
from os import listdir
from os.path import isfile, join

img_name = 'satellite' 
image_files = ['./'+img_name+'_images/' + f for f in listdir('./'+img_name+'_images/')]
images = [PIL.Image.open(x) for x in image_files]

edge_length_x = x_tile_range[1] - x_tile_range[0]
edge_length_y = y_tile_range[1] - y_tile_range[0]
edge_length_x = max(1,edge_length_x)
edge_length_y = max(1,edge_length_y)
width, height = images[0].size
width, height =  width // 4, height // 4


total_width = width*edge_length_x
total_height = height*edge_length_y



composite = PIL.Image.new('RGB', (total_width, total_height))
print(total_width,total_height)

anim_idx = 0
y_offset = 0
for i in range(0,edge_length_x):
    x_offset = 0
    for j in range(0,edge_length_y):
        tmp_img = PIL.Image.open('./'+img_name+'_images/' + str(i) + '.' + str(j) + '.png')
        m,n = tmp_img.size  # (200, 374)
        tmp_img = tmp_img.resize((int(m/4) , int(n/4)), Image.ANTIALIAS)
        composite.paste(tmp_img, (y_offset,x_offset))
        x_offset += width

        composite.save('./animate/'+str(anim_idx).zfill(4)+'.jpg',optimize=True,quality=25)
        anim_idx += 1
        print(anim_idx)

        
    y_offset += height


composite.save('./composite_images/'+img_name+'.png')




mkdir: cannot create directory ‘./composite_images’: File exists
mkdir: cannot create directory ‘./animate’: File exists
1152 1152
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [ ]:
import imageio
images = []
filenames = [f"animate/00{i}.jpg" if i >= 10 else f"animate/000{i}.jpg" for i in range(81)]
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('movie.gif', images)